In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io

In [2]:
response_X = rq.get('https://github.com/Rohan75/ConvolutionalAttention/blob/master/ConvolutionalAttentionNN/train_x.npy?raw=true')
response_X.raise_for_status()

response_y = rq.get('https://github.com/Rohan75/ConvolutionalAttention/blob/master/ConvolutionalAttentionNN/train_y.npy?raw=true')
response_y.raise_for_status()

X = np.load(io.BytesIO(response_X.content), allow_pickle=True)
y = np.load(io.BytesIO(response_y.content), allow_pickle=True)

In [12]:
input_layer = layers.Input(shape=(200, 4))

conv1 = layers.Conv1D(filters=200, kernel_size=13, activation='relu', padding='same', name='conv1')(input_layer)
pool1 = layers.MaxPool1D(pool_size=6)(conv1)

positions = tf.range(pool1.shape[1])
context = layers.Embedding(input_dim=pool1.shape[1], output_dim=pool1.shape[2])(positions)

contextual_meaning = tf.add(pool1, context)
inputs = layers.Dense(200)(contextual_meaning)

query = layers.Dense(200)(inputs)
value = layers.Dense(200)(inputs)

attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32)(query, value, return_attention_scores=True)
attention_inputs = tf.add(inputs, attention)

fcl1 = layers.Dense(256, activation='relu')(attention_inputs)
fcl2 = layers.Dense(256, activation='relu')(fcl1)

flat = layers.Flatten()(fcl2)

output = layers.Dense(1, activation='sigmoid')(flat)

In [13]:
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=25, validation_split=0.1)

Epoch 1/25
282/282 [==============================] - 15s 48ms/step - loss: 0.6887 - accuracy: 0.5146 - val_loss: 0.1309 - val_accuracy: 0.9710
Epoch 2/25
282/282 [==============================] - 13s 47ms/step - loss: 0.1322 - accuracy: 0.9655 - val_loss: 0.1090 - val_accuracy: 0.9750
Epoch 3/25
282/282 [==============================] - 15s 52ms/step - loss: 0.1139 - accuracy: 0.9727 - val_loss: 0.1058 - val_accuracy: 0.9740
Epoch 4/25
151/282 [===============>..............] - ETA: 6s - loss: 0.0923 - accuracy: 0.9778

KeyboardInterrupt: 